# Comprehensive Exploratory Data Analysis
## Data Breach Patterns and Business Insights

**Author:** T. Spivey  
**Course:** BUS 761  
**Assignment:** 5 - Exploratory Data Analysis Module  
**Date:** October 2025

---

## Executive Summary

This notebook demonstrates a comprehensive exploratory data analysis of 35,378 data breach incidents reported in the United States from 2003-2025. Using our newly developed **modular EDA package**, we uncover critical patterns in breach frequency, severity, and industry vulnerabilities.

### Key Findings:
1. **Industry-Specific Vulnerabilities**: Healthcare experiences 43% more disclosure breaches than expected
2. **Financial Sector Risk**: Physical breaches are 169% higher than expected in financial services
3. **Retail Targeting**: Payment card breaches in retail are 400% above statistical expectation
4. **Impact Relationships**: Non-linear relationship between total and resident impact (Spearman ρ=0.52 vs Pearson r=0.32)
5. **Temporal Trends**: Breach frequency and severity show distinct time-based patterns

---

## 1. Setup and Data Loading

First, we'll import our modular EDA package and load the data from our SQLite database.

In [1]:
# Import our custom EDA package
import sys
sys.path.append('..')  # Add parent directory to path

from eda_package import BreachAnalyzer, BreachVisualizer, DataLoader
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)

print("✓ Packages imported successfully")

✓ Packages imported successfully


In [2]:
# Initialize data loader
loader = DataLoader('../databreach.db')

# Load main breach dataset
df_breach = loader.load_breach_data()

print(f"Loaded {len(df_breach):,} breach records")
print(f"Columns: {df_breach.shape[1]}")
print(f"\nData Range: {df_breach['breach_date'].min()} to {df_breach['breach_date'].max()}")

Loaded 35,378 breach records
Columns: 20

Data Range: 1999-09-22 00:00:00 to 2025-05-17 00:00:00


In [3]:
# Display database summary
table_info = loader.get_table_info()
print("\nDatabase Tables:")
print("="*60)
for table, count in table_info.items():
    print(f"{table:40s}: {count:>10,} rows")


Database Tables:
databreach                              :     35,378 rows
sec_company_reference                   :     10,142 rows
correlation_results                     :          2 rows
chi_squared_summary                     :          1 rows
chi_squared_observed                    :          8 rows
chi_squared_expected                    :          8 rows
anova_results                           :          1 rows
tukey_hsd_results                       :         28 rows
descriptive_stats_by_org                :          8 rows
simple_regression_results               :          1 rows
multiple_regression_coefficients        :         13 rows
multiple_regression_results             :          1 rows
logistic_regression_results             :          1 rows
time_series_monthly                     :        223 rows
time_series_yearly                      :         22 rows


## 2. Data Overview and Quality Assessment

Let's examine the structure and quality of our dataset.

In [4]:
# Display first few records
print("Sample Records:")
df_breach.head()

Sample Records:


,id,org_name,reported_date,breach_date,end_breach_date,incident_details,information_affected,organization_type,breach_type,normalized_org_name,group_org_breach_type,group_org_type,total_affected,residents_affected,breach_location_street,breach_location_city,breach_location_state,breach_location_zip,breach_location_country,tags
0,0000aacf-a54b-57f0-a293-54ff1056d157,"Etz Hayim Holdings, SPC. d/b/a Lazarus Naturals",2020-12-11,2020-09-05,2020-09-14,The New Hampshire Attorney General's Office re...,"{""""Encryption Status"""":""""Unencrypted"""",""""Categ...",BSO,HACK,"Etz Hayim Holdings, SPC. d/b/a Lazarus Naturals",HACK,BSO,70.0,70.0,1116 Northwest 51st Street,Seattle,WA,98107,United States,90-days-or-longer-response
1,0003674d-b6eb-57e5-8a91-51815b397413,"Arrow Electronics, Inc.",2010-03-03,2010-02-18,NaT,The New Hampshire Department of Justice report...,"{""""Encryption Status"""":""""Unencrypted"""",""""Categ...",BSO,PORT,"Arrow Electronics, Inc.",PORT,BSO,4044.0,52.0,25 Hub Drive,Melville,NY,11747,United States,None
2,0005b2e4-1efd-5aea-ba68-44b371792fd7,"North Atlantic Telecom, Inc.",2013-05-08,NaT,NaT,The U.S. Department of Health and Human Servic...,"{""""Encryption Status"""":""""Unencrypted"""",""""Categ...",BSO,STAT,"North Atlantic Telecom, Inc.",STAT,BSO,539.0,NaN,UNKN,UNKN,TN,UNKN,United States,None
3,00088938-1c11-5554-800c-421bf4fd2416,LPL Financial,2011-09-23,NaT,NaT,The Massachusetts Office of Consumer Affairs a...,"{""""Encryption Status"""":""""Unencrypted"""",""""Categ...",BSF,PHYS,LPL Financial LLC,PHYS,BSF,NaN,1.0,UNKN,UNKN,UNKN,UNKN,UNKN,None
4,000a044e-c20f-5330-aa31-dca3a89020d1,"Bolton Global Capital, Inc.",2022-03-10,2021-08-12,2021-09-09,The Maryland Office of the Attorney General re...,"{""""Encryption Status"""": """"Unencrypted"""", """"Cat...",BSF,HACK,"Bolton Global Capital, Inc.",HACK,BSF,48.0,48.0,UNKN,UNKN,UNKN,UNKN,UNKN,"sensitive-personal-information,finance,persona..."


In [5]:
# Data quality assessment
print("\nData Quality Report")
print("="*60)
print(f"Total Records: {len(df_breach):,}")
print(f"\nMissing Values by Column:")
missing = df_breach.isnull().sum()
missing_pct = (missing / len(df_breach) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Percentage', ascending=False))


Data Quality Report
Total Records: 35,378

Missing Values by Column:
                    Missing Count  Percentage
tags                        20067       56.72
end_breach_date             18241       51.56
total_affected              16581       46.87
residents_affected          12769       36.09
breach_date                 12116       34.25
reported_date                  84        0.24


In [6]:
# Categorical variable distributions
print("\nOrganization Type Distribution:")
print(df_breach['organization_type'].value_counts())
print("\n" + "="*60)
print("\nBreach Type Distribution:")
print(df_breach['breach_type'].value_counts())


Organization Type Distribution:
organization_type
BSO     11346
MED     10039
BSF      5391
BSR      2883
EDU      1911
NGO      1727
UNKN     1098
GOV       983
Name: count, dtype: int64


Breach Type Distribution:
breach_type
HACK    25792
PHYS     3629
DISC     3062
PORT     1530
INSD     1039
STAT      211
CARD      115
Name: count, dtype: int64


## 3. Statistical Analysis

Now we'll use our `BreachAnalyzer` class to conduct comprehensive statistical analyses.

In [7]:
# Initialize analyzer
analyzer = BreachAnalyzer(df_breach, alpha=0.05)
print(f"Analyzer initialized: {analyzer}")

Analyzer initialized: BreachAnalyzer(records=35378, alpha=0.05, analyses=0)


### 3.1 Descriptive Statistics

First, let's examine the central tendencies and distributions of key numeric variables.

In [8]:
# Overall descriptive statistics
desc_stats_overall = analyzer.descriptive_statistics()
print("\nOverall Descriptive Statistics:")
print("="*80)
desc_stats_overall


Overall Descriptive Statistics:


,variable,count,mean,median,std,min,max,q25,q75,skewness,kurtosis
0,total_affected,18797,430791.059,1338.0,1.270e+07,0.0,1.000e+09,66.0,9302.0,49.667,2943.863
1,residents_affected,22609,2344.153,5.0,4.277e+04,0.0,3.244e+06,1.0,51.0,48.863,3073.789


In [9]:
# Descriptive statistics by organization type
desc_stats_by_org = analyzer.descriptive_statistics(group_by='organization_type')
print("\nDescriptive Statistics by Organization Type:")
print("="*80)
desc_stats_by_org


Descriptive Statistics by Organization Type:


,count,mean,median,std,min,max,q25,q75,iqr
organization_type,,,,,,,,,
BSF,1971,6.263e+05,300.0,8.528e+06,0.0,1.455e+08,5.0,5308.00,5303.00
BSO,5628,7.712e+05,614.0,1.941e+07,0.0,1.000e+09,12.0,5665.25,5653.25
BSR,1182,1.386e+06,435.5,2.519e+07,1.0,5.000e+08,20.0,4858.75,4838.75
EDU,907,2.505e+04,670.0,1.280e+05,1.0,1.265e+06,10.5,5086.50,5076.00
GOV,397,7.853e+04,1100.0,4.580e+05,1.0,4.663e+06,15.0,9500.00,9485.00
MED,7454,1.075e+05,3248.5,1.455e+06,1.0,8.000e+07,941.0,17896.00,16955.00
NGO,826,2.258e+04,468.0,1.476e+05,1.0,3.300e+06,7.0,5038.25,5031.25
UNKN,432,2.352e+04,105.5,2.040e+05,0.0,3.100e+06,3.0,1673.25,1670.25


**Business Insight:** The data shows significant skewness (positive skewness values), indicating that most breaches are relatively small, but a few massive breaches drive up the mean. The median provides a better measure of "typical" breach size.

### 3.2 Correlation Analysis

Examine the relationship between total individuals affected and state residents affected.

In [10]:
# Correlation analysis
corr_results = analyzer.correlation_analysis()

print("\nCorrelation Analysis Results:")
print("="*80)
print(f"Variables: {corr_results['variable_1']} vs {corr_results['variable_2']}")
print(f"Sample Size: {corr_results['sample_size']:,} valid pairs\n")

print(f"Pearson Correlation (Linear):")
print(f"  r = {corr_results['pearson_r']:.4f}")
print(f"  p-value = {corr_results['pearson_p']:.6f}")
print(f"  Significant: {corr_results['pearson_significant']}\n")

print(f"Spearman Correlation (Monotonic):")
print(f"  ρ = {corr_results['spearman_rho']:.4f}")
print(f"  p-value = {corr_results['spearman_p']:.6f}")
print(f"  Significant: {corr_results['spearman_significant']}")


Correlation Analysis Results:
Variables: total_affected vs residents_affected
Sample Size: 11,555 valid pairs

Pearson Correlation (Linear):
  r = 0.3150
  p-value = 0.000000
  Significant: True

Spearman Correlation (Monotonic):
  ρ = 0.5167
  p-value = 0.000000
  Significant: True


**Key Finding:** The difference between Pearson (r=0.32) and Spearman (ρ=0.52) suggests:
- Strong monotonic relationship (rank-order)
- Non-linear pattern in actual values
- Presence of outliers affecting linear correlation
- Typical in breach data where mega-breaches distort linear measures

### 3.3 Chi-Squared Test: Industry vs Breach Type

Test whether organization type and breach type are independent.

In [11]:
# Chi-squared test
chi_results = analyzer.chi_squared_test('organization_type', 'breach_type')

print("\nChi-Squared Test Results:")
print("="*80)
print(f"Null Hypothesis: {chi_results['variable_1']} and {chi_results['variable_2']} are independent")
print(f"\nχ² Statistic: {chi_results['chi2_statistic']:.2f}")
print(f"p-value: {chi_results['p_value']:.8f}")
print(f"Degrees of Freedom: {chi_results['degrees_of_freedom']}")
print(f"Sample Size: {chi_results['sample_size']:,}")
print(f"\nConclusion: {'REJECT' if chi_results['significant'] else 'FAIL TO REJECT'} null hypothesis")
print(f"Interpretation: {'Strong evidence of' if chi_results['significant'] else 'No significant'} relationship")


Chi-Squared Test Results:
Null Hypothesis: organization_type and breach_type are independent

χ² Statistic: 5069.93
p-value: 0.00000000
Degrees of Freedom: 42
Sample Size: 35,378

Conclusion: REJECT null hypothesis
Interpretation: Strong evidence of relationship


In [12]:
# Display observed frequencies
print("\nObserved Frequencies (Contingency Table):")
chi_results['observed']


Observed Frequencies (Contingency Table):


breach_type,CARD,DISC,HACK,INSD,PHYS,PORT,STAT
organization_type,,,,,,,
BSF,55,479,2963,252,1489,130,23
BSO,8,774,9542,258,340,402,22
BSR,45,51,2636,47,69,31,4
EDU,0,218,1521,22,67,80,3
GOV,2,162,576,20,195,20,8
MED,0,1243,6325,397,1130,794,150
NGO,5,92,1450,31,105,43,1
UNKN,0,43,779,12,234,30,0


In [13]:
# Calculate deviations from expected
deviation = chi_results['observed'] - chi_results['expected']
deviation_pct = (deviation / chi_results['expected'] * 100).round(1)

print("\nDeviation from Expected (Percentage):")
print("Positive = More breaches than expected, Negative = Fewer")
deviation_pct


Deviation from Expected (Percentage):
Positive = More breaches than expected, Negative = Fewer


breach_type,CARD,DISC,HACK,INSD,PHYS,PORT,STAT
organization_type,,,,,,,
BSF,213.9,2.7,-24.6,59.2,169.3,-44.2,-28.5
BSO,-78.3,-21.2,15.4,-22.6,-70.8,-18.1,-67.5
BSR,380.2,-79.6,25.4,-44.5,-76.7,-75.1,-76.7
EDU,-100.0,31.8,9.2,-60.8,-65.8,-3.2,-73.7
GOV,-37.4,90.4,-19.6,-30.7,93.4,-53.0,36.5
MED,-100.0,43.1,-13.6,34.7,9.7,82.9,150.5
NGO,-10.9,-38.5,15.2,-38.9,-40.7,-42.4,-90.3
UNKN,-100.0,-54.8,-2.7,-62.8,107.8,-36.8,-100.0


**Critical Business Insights:**

The significant chi-squared result (p < 0.001) confirms that different industries face different breach threats:

1. **Healthcare (MED)**: 43% more DISC (disclosure) breaches → Focus on access controls
2. **Financial (BSF)**: 169% more PHYS (physical) breaches → Strengthen document security
3. **Retail (BSR)**: 400% more CARD breaches → Enhanced POS security critical
4. **Business/Other (BSO)**: 15% more HACK attacks → Cyber defense priority

### 3.4 ANOVA: Breach Impact Across Industries

Test whether breach severity differs significantly across organization types.

In [14]:
# ANOVA test
anova_results = analyzer.anova_test('organization_type', 'total_affected')

print("\nANOVA Results:")
print("="*80)
print(f"Question: Does breach impact vary across organization types?")
print(f"\nF-Statistic: {anova_results['f_statistic']:.4f}")
print(f"p-value: {anova_results['p_value']:.6f}")
print(f"Number of Groups: {anova_results['n_groups']}")
print(f"\nConclusion: {'YES' if anova_results['significant'] else 'NO'} - ")
print(f"Breach impact {'DOES' if anova_results['significant'] else 'DOES NOT'} vary significantly by industry")


ANOVA Results:
Question: Does breach impact vary across organization types?

F-Statistic: 2.6535
p-value: 0.009653
Number of Groups: 8

Conclusion: YES - 
Breach impact DOES vary significantly by industry


In [15]:
# Display group statistics
print("\nGroup Statistics:")
anova_results['group_statistics'].sort_values('mean', ascending=False)


Group Statistics:


,mean,median,std,count
organization_type,,,,
BSR,1.386e+06,435.5,2.519e+07,1182
BSO,7.712e+05,614.0,1.941e+07,5628
BSF,6.263e+05,300.0,8.528e+06,1971
MED,1.075e+05,3248.5,1.455e+06,7454
GOV,7.853e+04,1100.0,4.580e+05,397
EDU,2.505e+04,670.0,1.280e+05,907
UNKN,2.352e+04,105.5,2.040e+05,432
NGO,2.258e+04,468.0,1.476e+05,826


### 3.5 Linear Regression: Predicting Resident Impact

Build a simple linear model to predict resident impact from total individuals affected.

In [16]:
# Simple linear regression
reg_results = analyzer.simple_linear_regression()

print("\nSimple Linear Regression Results:")
print("="*80)
print(f"Model: {reg_results['y_variable']} = {reg_results['slope']:.6f} * {reg_results['X_variable']} + {reg_results['intercept']:.2f}")
print(f"\nR² (Variance Explained): {reg_results['r_squared']:.4f}")
print(f"Sample Size: {reg_results['sample_size']:,}")
print(f"\nInterpretation: {reg_results['r_squared']*100:.1f}% of variance in resident impact")
print(f"                is explained by total individuals affected")


Simple Linear Regression Results:
Model: residents_affected = 0.002739 * total_affected + 3193.34

R² (Variance Explained): 0.0992
Sample Size: 11,555

Interpretation: 9.9% of variance in resident impact
                is explained by total individuals affected


**Model Interpretation:**
- Slope ≈ 0.0027: For every 1,000 total individuals affected, ~2.7 additional residents are affected
- Low R² (0.099): Simple linear model explains only 10% of variance
- Suggests: Need more complex model or additional predictors

### 3.6 Time Series Analysis

Analyze breach trends over time.

In [17]:
# Time series by year
time_series = analyzer.time_series_analysis(freq='Y')

print("\nTime Series Analysis (Yearly):")
print("="*80)
time_series.tail(10)  # Show last 10 years


Time Series Analysis (Yearly):


,period,breach_count,total_affected_sum,total_affected_mean
14,2016,1319,1.821e+07,3.627e+04
15,2017,1388,6.092e+08,1.012e+06
16,2018,1459,1.534e+09,2.662e+06
17,2019,1421,6.796e+06,1.497e+04
18,2020,2609,4.593e+07,4.029e+04
19,2021,3189,1.444e+08,8.553e+04
20,2022,2533,3.734e+07,2.818e+04
21,2023,4707,3.297e+08,1.442e+05
22,2024,2519,1.020e+08,9.213e+04
23,2025,144,5.400e+05,1.000e+04


### 3.7 Logistic Regression: Predicting Severe Breaches

Classify breaches as severe (>10,000 affected) or non-severe.

In [18]:
# Logistic regression
logit_results = analyzer.logistic_regression_severity(threshold=10000)

print("\nLogistic Regression Results:")
print("="*80)
print(f"Classification Threshold: {logit_results['threshold']:,} individuals")
print(f"Model Accuracy: {logit_results['accuracy']:.2%}")
print(f"\nSevere Breaches: {logit_results['severe_count']:,}")
print(f"Non-Severe Breaches: {logit_results['non_severe_count']:,}")
print(f"\nTop Coefficients (Highest Risk Factors):")

coef_df = pd.DataFrame.from_dict(logit_results['coefficients'], 
                                 orient='index', columns=['Coefficient'])
coef_df.sort_values('Coefficient', ascending=False).head(10)


Logistic Regression Results:
Classification Threshold: 10,000 individuals
Model Accuracy: 76.38%

Severe Breaches: 4,439
Non-Severe Breaches: 14,358

Top Coefficients (Highest Risk Factors):


,Coefficient
organization_type_MED,0.911
breach_type_HACK,0.777
organization_type_GOV,0.413
organization_type_BSO,-0.087
organization_type_EDU,-0.214
organization_type_BSR,-0.256
organization_type_NGO,-0.283
breach_type_STAT,-0.317
breach_type_INSD,-0.353
breach_type_PORT,-0.410


## 4. Data Visualization

Now we'll use our `BreachVisualizer` class to create publication-quality visualizations.

In [19]:
# Initialize visualizer
viz = BreachVisualizer(df_breach, output_dir='output/visualizations')
print(f"Visualizer initialized: {viz}")

Visualizer initialized: BreachVisualizer(output_dir='output/visualizations', figures=0)


In [ ]:
# Generate comprehensive dashboard
# Load necessary data for visualizations
chi_observed = loader.load_statistical_results('chi_squared_observed')
time_series_data = loader.load_statistical_results('time_series_yearly')

# Prepare descriptive stats for visualization
desc_for_viz = desc_stats_by_org.reset_index()
desc_for_viz.columns = ['organization_type'] + desc_for_viz.columns[1:].tolist()

# Create all visualizations
viz.create_comprehensive_dashboard(
    chi_observed=chi_observed,
    desc_stats=desc_for_viz,
    time_series=time_series_data,
    correlation_stats=corr_results,
    regression_results=reg_results
)

GENERATING COMPREHENSIVE VISUALIZATION DASHBOARD

Saved: output/visualizations\1_industry_vulnerability_heatmap.png
Saved: output/visualizations\2_breach_frequency.png
Saved: output/visualizations\3_impact_correlation.png
Saved: output/visualizations\4_sector_impact.png
Saved: output/visualizations\5_time_series_trends.png
Saved: output/visualizations\6_regression_fit.png

DASHBOARD COMPLETE
Location: output/visualizations
Total visualizations: 6


## 5. Business Insights Summary

Let's generate actionable business insights from our analyses.

In [ ]:
# Get business insights
insights = analyzer.get_business_insights()

print("\nBUSINESS INSIGHTS")
print("="*80)
for category, insight in insights.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    print(f"  {insight}")

## 6. Strategic Recommendations

Based on our comprehensive analysis, here are the key strategic recommendations:

### For Healthcare Organizations (MED):
- **Priority:** Disclosure prevention
- **Action:** Implement stricter access controls and data handling procedures
- **Rationale:** 43% more disclosure breaches than expected

### For Financial Services (BSF):
- **Priority:** Physical document security
- **Action:** Enhanced document destruction protocols, secure storage
- **Rationale:** 169% more physical breaches than expected

### For Retail (BSR):
- **Priority:** Payment card security
- **Action:** POS system hardening, EMV compliance, fraud detection
- **Rationale:** 400% more card breaches than expected

### For All Organizations:
- **Trend Monitoring:** Breach frequency and severity evolving over time
- **Impact Planning:** Most breaches are small, but extreme outliers drive total impact
- **Resource Allocation:** Industry-specific security investments provide better ROI

---

## 7. Conclusion

This exploratory data analysis has revealed significant patterns in data breach vulnerabilities across industries. Our modular EDA package enables:

1. **Reusable Analysis:** Object-oriented design allows easy replication with new data
2. **Statistical Rigor:** Multiple hypothesis tests confirm significant relationships
3. **Business Focus:** Insights directly inform security investment decisions
4. **Visualization Quality:** Publication-ready charts for stakeholder communication

### Next Steps:
- Develop predictive models (Assignment 6)
- Create interactive dashboard (Assignment 7)
- Monitor trends with updated data
- Industry-specific deep dives

---

**Contact:** T. Spivey | BUS 761 | October 2025